Goal was to try and compare appoaches for text cleaning (nltk and textasy) and try translation models for workflow

Result - nltk and textacy clean text approxinately the same for this particular text, textacy have more subfunctions to drop urls, emails, phone numbers if needed in the future. And dictionary of textacy stop keywords is bigger, so its better to take this, because further texts may have more words which need to be cleaned.


In [ ]:
#pip install --user spacy

In [ ]:
#pip install --user textacy

In [ ]:
#pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0.tar.gz


  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0.tar.gz (14.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for spacy<3.8.0,>=3.7.0 from https://files.pythonhosted.org/packages/39/e1/08681583569f435347ced0535b27c073fcc9a927d9b4293c963092f2d01c/spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata
  Using cached spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Obtaining dependency information for thinc<8.3.0,>=8.2.2 from https://files.pythonhosted.org/packages/5e/0e/5e7b24e046e0725eafc37ded0cd9bfaf789efb894101a7aca8a73dba81de/thinc-8.2.5-cp311-cp311-win_amd64.whl.metadata
  Using cached thinc-8.2.5-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached spacy-3.7.5-cp311-cp311-win_amd64.whl (12.1 MB)
Using cached thinc-8.2.5-cp311-cp311-win_amd64.whl (1.5 MB)
  Created wheel for de-core-news-sm: filename=de_core_news_sm-3.7

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\Екатерина\\AppData\\Roaming\\Python\\Python311\\site-packages\\~~inc\\backends\\cblas.cp311-win_amd64.pyd'
Check the permissions.



In [ ]:
import json
import requests
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import textacy.preprocessing.normalize as tnorm
import textacy.preprocessing.remove as trem
import spacy
import textacy.preprocessing as tp
from nltk.tokenize import word_tokenize
from transformers import MarianMTModel, MarianTokenizer
import torch
nlp = spacy.load("de_core_news_sm")
nltk.download('stopwords')
nltk.download('punkt')
german_stopwords = set(stopwords.words('german'))
OLLAMA_ENDPOINT ="http://127.0.0.1:11434/api/generate"

In [ ]:
df_german = pd.df_german = pd.read_json('data/uksh_2.json')

transcripts = df_german["results"]["transcripts"]
items = df_german["results"]["items"]

df_transcripts = pd.DataFrame(transcripts)

df_items = pd.DataFrame([
    {
        "transcript": item["alternatives"][0]["content"],
        "startTime": item.get("start_time", None),
        "endTime": item.get("end_time", None)
    }
    for item in items if item["type"] == "pronunciation"
])

print(df_items.head())

  transcript startTime endTime
0       Test      0.14   0.579
1       Test     0.589     1.1
2     record      2.75   3.549
3         62     3.559   4.489
4        der       4.5   4.599


In [ ]:
#text_file_g = open('text_german.txt', 'w', encoding='utf-8')
#text_file_g.write(text_german)
#text_file_g.close()

In [ ]:
# Open already prepared
with open('data/text_german.txt', 'r', encoding='utf-8') as file:
    text_german = file.read()

print(text_german)

Test Test record 62 der Frequenz Wir sind in der Vorbereitung der Prozedur Das macht sorry findet sie noch nicht Also an die Ki wir sind noch noch nicht mal am Tisch Video funktioniert leider nicht hoffentlich funktioniert der Ton können Sie es mal Ich hab mal meine meine heutigen Handys hier hingelegt Ja die vier Stück na bist du schon im Weihnachten Habt ihr auch diese Kita App family App bei euch in der Kita Ja ja auch ja ist praktisch ne das find ich auch Ja ja ja ja und dann guckst du rein Wirsing kohl mit apfelgelee Kroketten mal beeindruckend wenn sie es war so lecker in der in der Kita heute und dann guckst du rein es gab Brokkoli mit äh ja bei uns zu Hause dann ja Mama der Kartoffelbrei in der Kita der ist so gut der Massen zu Hause Kartoffel die beste köchin appetit zu essen was sie einfach ja ja ach ja das ist so viel witzig muss bitte den freier von einer un das sorry das ich finde das richtig geil danke wenn das ihr das so sagt weil das meine ich ernst der braucht viel und

In [ ]:
len(text_german) #actual lenght without cleaning

34698

## Comparison textacy and nltk

Here we providing stop words which will be deleted from the text in German lanuage for NLTK and textacy. Both NLTK and textacy have dictionaries of stop words in different languages, so they are not sticking only for English.

In [ ]:
german_stopwords = list(nlp.Defaults.stop_words)

german_stopwords[:10] #here there are words which will be remove to show

['indem',
 'müsst',
 'meinen',
 'neunten',
 'du',
 'gewollt',
 'dem',
 'vierten',
 'wart',
 'allein']

In [ ]:
#clean text using nltk
def clean_text(text):
    words = nltk.word_tokenize(text, language='german')
    cleaned_text = " ".join(word for word in words if word.lower() not in german_stopwords)
    return cleaned_text

In [ ]:
#clean text using textacy
def clean_text_textacy(text):
    text = tnorm.whitespace(text)

    text = text = trem.punctuation(text)

    doc = nlp(text)
    german_stopwords = nlp.Defaults.stop_words
    cleaned_text = " ".join([token.text for token in doc if token.text.lower() not in german_stopwords])

    cleaned_text = cleaned_text.strip()

    return cleaned_text

In [ ]:
# Clean
clean_nltk = clean_text(text_german)
clean_textacy = clean_text_textacy(text_german)

# Compare sets
words_original = set(text_german.split())
words_nltk = set(clean_nltk.split())
words_textacy = set(clean_textacy.split())

# Find words removed by each method
removed_nltk = words_original - words_nltk
removed_textacy = words_original - words_textacy

print("Words removed by nltk:", removed_nltk)
print("Words removed by textacy:", removed_textacy)




Words removed by nltk: {"nimm's", 'meinen', 'Na', 'du', 'Noch', 'dem', 'doch', 'denen', 'Wir', 'einen', 'könnte', 'ein', 'lange', 'Nein', 'habe', 'erst', 'ab', 'Sehr', 'Tagen', 'denn', 'unser', "mach's", 'los', 'werden', 'einer', 'ja', 'an', 'morgen', 'Schon', 'Am', 'leider', 'Keine', 'keinem', 'kleiner', 'Ein', 'keiner', 'Macht', 'wurde', 'möchte', 'acht', 'noch', 'wäre', 'zweite', 'zum', 'unter', 'darauf', 'solche', 'ich', 'gerade', "wird's", 'erste', 'dich', 'musste', 'na', 'keine', 'Also', 'gewesen', 'Eine', 'kam', 'Vielleicht', 'macht', 'zwei', 'unsere', 'sei', 'ach', 'auch', 'vor', 'tag', 'Jahren', 'immer', 'wollen', 'sind', 'selbst', 'Dann', 'Dafür', 'diesen', 'besser', 'hat', 'gab', 'wenn', 'Aber', 'heißt', 'Kannst', 'euch', 'beiden', 'sechs', 'Bin', 'dafür', 'jahr', 'Sie', 'sondern', 'in', 'sehr', 'oder', 'nach', 'her', 'bei', 'mögt', "seh's", 'vier', 'große', 'ganze', 'Es', 'uns', 'hin', 'Alles', 'wieder', 'einem', 'sagt', 'kannst', 'Wie', 'vielleicht', 'zur', 'Kam', 'den', '

Here we are showing lenght of text after cleaning both of approaches. Difference is only in time references pronounced in the video (nltk keep 13:15 or 13:43 while textacy removed them, but its not from timestamps by the model, its only from text)

In [ ]:
len(clean_textacy)

14862

In [ ]:
len(clean_nltk)

14879

Half is reduced! Lets see the statistics of comparison of dictionaries in german

In [ ]:
nltk_stopwords = set(stopwords.words('german'))
textacy_stopwords = nlp.Defaults.stop_words

common_stopwords = nltk_stopwords & textacy_stopwords
nltk_only = nltk_stopwords - textacy_stopwords
textacy_only = textacy_stopwords - nltk_stopwords

print(f"Total NLTK stopwords: {len(nltk_stopwords)}")
print(f"Total Textacy/SpaCy stopwords: {len(textacy_stopwords)}")
print(f"Common stopwords: {len(common_stopwords)}")
print(f"Unique to NLTK: {len(nltk_only)}")
print(f"Unique to Textacy/SpaCy: {len(textacy_only)}")


Total NLTK stopwords: 232
Total Textacy/SpaCy stopwords: 543
Common stopwords: 208
Unique to NLTK: 24
Unique to Textacy/SpaCy: 335


Textacy is more rich in terms of dictionary. But for german text they are same words. Also with textacy after we can remove URLs, emails and phone numbers. With textacy its possible to remove all numbers, I think we can do it because we will measure time with timestamps, not conversational mention of time, so its useless. Lets compare English.

In [ ]:
english_stopwords = set(stopwords.words('english'))

In [ ]:
print(len(english_stopwords))
english_stopwords_list = list(english_stopwords)
print(english_stopwords_list[:10])   # to have a view

198
['our', 'ours', 'mustn', 'over', 'should', 'to', 'wasn', 'couldn', 'o', 'because']


In [ ]:
nltk_stopwords = set(stopwords.words('english'))
textacy_stopwords = nlp.Defaults.stop_words

common_stopwords = nltk_stopwords & textacy_stopwords
nltk_only = nltk_stopwords - textacy_stopwords
textacy_only = textacy_stopwords - nltk_stopwords

print(f"Total NLTK stopwords: {len(nltk_stopwords)}")
print(f"Total Textacy stopwords: {len(textacy_stopwords)}")
print(f"Common stopwords: {len(common_stopwords)}")
print(f"Unique to NLTK: {len(nltk_only)}")
print(f"Unique to Textacy: {len(textacy_only)}")

Total NLTK stopwords: 198
Total Textacy stopwords: 543
Common stopwords: 8
Unique to NLTK: 190
Unique to Textacy: 535


Because of Textacy has more wider dictionary of stopwords we suggest to use Textacy because future videos can contain more words which are frequent in the human's speech even in this particular text they clean approximately same amount of words.

## Translation

In [ ]:
# Open already prepared clean text
with open('data/cleaned_text.txt', 'r', encoding='utf-8') as file:
    cleaned_text = file.read()

print(cleaned_text)

Test Test record 62 Frequenz Vorbereitung Prozedur macht sorry findet Ki mal Tisch Video funktioniert leider hoffentlich funktioniert Ton mal mal heutigen Handys hingelegt Ja vier Stück na schon Weihnachten Habt Kita App family App Kita Ja ja ja praktisch ne find Ja ja ja ja guckst rein Wirsing kohl apfelgelee Kroketten mal beeindruckend lecker Kita heute guckst rein gab Brokkoli äh ja Hause ja Mama Kartoffelbrei Kita gut Massen Hause Kartoffel beste köchin appetit essen einfach ja ja ach ja witzig bitte freier un sorry finde richtig geil danke sagt ernst braucht fahr schon bisschen gerade schon mal bisschen fahren tatsächlich genau 25 Uhr 35 zwei immer ja ecke Zero gravity beziehen ja schön Nee sag ja gar Sag mal betonen erst komme 's lernen immer geweiht 's ruhig mal lernen musst mal anleiten Tasche musst hand genau besten beiden händen möchte gerne mal lehr möchte gerne lernen ja darauf ziemlich idiotensicher Schon mal ja heißt gehst Hand rein rein okay ja ja ja ah ja richtig ja R R

### icky/translate

Because of model to translate has limit on input tokens, we decided to build split_text_into_chunks function which splits long german into smaller chunks. After we sent request to the endpoint of ollama and escpecially to this small translation model, which is built on the Gemma2 architecture, a lightweight neural network optimized for translation tasks. It has 2.61 billion parameters.

In [ ]:
# chunks
def split_text_into_chunks(text, chunk_length=400):
    chunks = []
    while len(text) > chunk_length:
        split_index = text.rfind(' ', 0, chunk_length)
        if split_index == -1:
            split_index = chunk_length
        chunks.append(text[:split_index])
        text = text[split_index:].strip()
    chunks.append(text)
    return chunks

# translation
def translate_chunks_icky(chunks, model="icky/translate", endpoint="http://127.0.0.1:11434/api/generate"):
    translated_chunks = []

    for chunk in chunks:
        prompt = f"Translate the following text from German to English:\n\n{chunk}"

        payload = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(endpoint, json=payload)
            if response.status_code == 200:
                result = response.json()
                translated_chunks.append(result["response"])
            else:
                print(f"Error {response.status_code}: {response.text}")
                translated_chunks.append("")

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            translated_chunks.append("")

    return translated_chunks


In [ ]:
chunks = split_text_into_chunks(cleaned_text, chunk_length=400)
chunks

['Test Test record 62 Frequenz Vorbereitung Prozedur macht sorry findet Ki mal Tisch Video funktioniert leider hoffentlich funktioniert Ton mal mal heutigen Handys hingelegt Ja vier Stück na schon Weihnachten Habt Kita App family App Kita Ja ja ja praktisch ne find Ja ja ja ja guckst rein Wirsing kohl apfelgelee Kroketten mal beeindruckend lecker Kita heute guckst rein gab Brokkoli äh ja Hause ja',
 "Mama Kartoffelbrei Kita gut Massen Hause Kartoffel beste köchin appetit essen einfach ja ja ach ja witzig bitte freier un sorry finde richtig geil danke sagt ernst braucht fahr schon bisschen gerade schon mal bisschen fahren tatsächlich genau 25 Uhr 35 zwei immer ja ecke Zero gravity beziehen ja schön Nee sag ja gar Sag mal betonen erst komme 's lernen immer geweiht 's ruhig mal lernen musst mal",
 "anleiten Tasche musst hand genau besten beiden händen möchte gerne mal lehr möchte gerne lernen ja darauf ziemlich idiotensicher Schon mal ja heißt gehst Hand rein rein okay ja ja ja ah ja rich

In [ ]:
translated_chunks = translate_chunks_icky(chunks)
translated_chunks

["Test Test record 62 Frequency preparation Procedure is sorry finds Ki mal table Video works unfortunately hopefully works Ton mal mal today's cell phones put down Yes four pieces na already Christmas We have Kita App family app Kita yes yes yes practically find Ja ja ja look inside Wirsing Kohl apple jelly Kroketten good impressive delicious Kita today look inside Brokkoli äh yes at home \n",
 "Mama potato mash good Kita well Mass home potato best chef appettit eat easy yes yes ok funny please free un sorry find right cool thanks says ernst needs to drive soon just a little already went for a little drive actually exactly 25 Uhr 35 two always yes corner zero gravity get it nice no say yes really emphasize first I'll learn  always dedicated learning gotta learn \n",
 "I need to teach you.  My hands are perfect, I want to learn, I would like to try it, I am sure. Just go in! Okay, okay, yes, yes, ah yes, right, yes. It's wrong, do it wrong, yes. Let’s get started. \n  \nLet's try that 

The model's response contains additional unnecessary comments in addition to the text, so we'll clean it up.

In [ ]:
def extract_text_inside_quotes(chunk_list):
    cleaned_chunks = []


    for chunk in chunk_list:
        matches = re.findall(r'"(.*?)"', chunk, flags=re.DOTALL)
        cleaned_chunks.extend(matches)

    combined_text = " ".join(cleaned_chunks)

    return combined_text


In [ ]:
cleaned_text_icky = extract_text_inside_quotes(translated_chunks)
print(cleaned_text_icky)

I think, it looks really cool. Alright, let's get started. I will go to bed. You can take off your clothes. Yes! Okay, okay, I hear you loud and clear. Yes, we are almost there. We have just the thing...  Jule, Charlotte... Just plug in that little connector…. Okay, okay, the whole thing is a mission and it's already been started at 26:00 pm. Okay. Seems like a decent jump. The needle is far back. It's fair. At 13:26, I think. Okay. Mr. Parin, a hundred kilos, one thousand units. Please connect the cables. One thousand units. Mr. Parin. Let him go in. Wait. He’s done. Marcell can do it. Yes. It’s recorded. No damn it. Let me just give it a bit of air. Ventil open, thanks! First things first, sorry, just a moment. Yes, I think so. Sonde, yes I saw it, it seems like a direct bad one, yeah you can go back a little bit. Just go there. Right now it's quite deep down, almost...  You know, it looks nice and small. 31, let me know! It kinda looks off. Maybe 3123, top yellow rock, looking good.

The model couldn't translate some parts of the text, so we'll try another, larger model.

### dorian 2b model

The dorian2b/vera is a fine-tuned version of Qwen2.5, designed for translation, language understanding, and mid-complex tasks. We used vesrion with 7b parameters because we had enough memory to download it, but for better quality we suggest to use bigger one

In [ ]:
# translation
def translate_chunks(chunks, model="dorian2b/vera", endpoint="http://127.0.0.1:11434/api/generate"):
    translated_chunks = []

    for chunk in chunks:
        prompt = f"Translate the following text from German to English:\n\n{chunk}"

        payload = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(endpoint, json=payload)
            if response.status_code == 200:
                result = response.json()
                translated_chunks.append(result["response"])
            else:
                print(f"Error {response.status_code}: {response.text}")
                translated_chunks.append("")

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            translated_chunks.append("")

    return translated_chunks


In [ ]:
translated_chunks2 = translate_chunks(chunks)
translated_chunks2

['Here\'s the translation of the German text into English:\n\n"Test Test record 62 Frequency Preparation Procedure makes sorry finds Ki on table Video works, but hopefully the sound will work too today Handset put down already four pieces now for Christmas Have Kita app family app Kita yes yes practical I see look in Wirsing cabbage apple jam Croquettes quite impressive tasty Kita today you can look in gave broccoli yeah home"',
 'Sure, here is the translation of the text:\n\n"Mom\'s potato soup, kindergarten good, mass at home, potatoes best cook, appetite eating easy yes yes ah yes funny please free and sorry find really great. Thank you says serious need to drive already a little bit just a little bit driving actually exactly 25:35 two always yes corner Zero gravity take yes nice No say yes really say once learn always consecrated always calm learn must sometimes"',
 'The translation of the given German text is:\n\n"teach bag you must hand exactly best both hands want to teach want 

In [ ]:
cleaned_text_dor = extract_text_inside_quotes(translated_chunks2)
print(cleaned_text_dor)

Test Test record 62 Frequency Preparation Procedure makes sorry finds Ki on table Video works, but hopefully the sound will work too today Handset put down already four pieces now for Christmas Have Kita app family app Kita yes yes practical I see look in Wirsing cabbage apple jam Croquettes quite impressive tasty Kita today you can look in gave broccoli yeah home Mom's potato soup, kindergarten good, mass at home, potatoes best cook, appetite eating easy yes yes ah yes funny please free and sorry find really great. Thank you says serious need to drive already a little bit just a little bit driving actually exactly 25:35 two always yes corner Zero gravity take yes nice No say yes really say once learn always consecrated always calm learn must sometimes teach bag you must hand exactly best both hands want to teach want to learn yeah pretty sure on that okay yeah yeah yeah ah yeah right yeah R R R r r do it wrong already do it wrong yeah get it okay really simple Yeah something fits eggs

This model shows more contextual understanding, so its better than previous one, but works longer

## Helsinki-NLP/opus-mt-de-en

The Helsinki-NLP/opus-mt-de-en is a model developed by the Language Technology Research Group at the University of Helsinki. It is designed for German-to-English translation and text-to-text generation, leveraging a transformer-based architecture. They also have anothe languages models

In [ ]:
model_name = "Helsinki-NLP/opus-mt-de-en"  # German to English model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

#GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def translate_text_marian(text, model=model, tokenizer=tokenizer, device=device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    outputs = model.generate(
        inputs["input_ids"],
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

def translate_chunks_marian(chunks, model=model, tokenizer=tokenizer, device=device):
    translated_chunks = []

    for chunk in chunks:
        translated_text = translate_text_marian(chunk['text'], model, tokenizer, device)

        translated_chunks.append({
            "translated_text": translated_text,
            "start_time": chunk["start_time"],
            "end_time": chunk["end_time"]
        })

    return translated_chunks


In [ ]:
translated_chunks3 = translate_chunks_marian(chunks)
for chunk in translated_chunks3:
    print(chunk)

Works very long, dont need to install model locally with ollama